In [1]:
from baza_mip import mip_session, sessionmaker
from baza_mip.models import Procesy_Przydzielone, Procesy
from sqlalchemy import create_engine, text
from Plan_Pracy.plan_pracy import kj_paczki

import pandas as pd


gen_engine = create_engine("sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/GENERATORY.db", echo=False)



In [2]:
#dostawy
with gen_engine.begin() as conn:
    suma_paczek = conn.execute(text("select SUM(ILE_ZAMOWIONE) from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 17_24'")).fetchall()[0][0]
    zamowione_paczki = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 17_24'")).fetchall()
    owaty = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 17_24' AND MODEL == 'WILLOW'")).fetchall()
    owaty_mb = conn.execute(text("select * from OWATY")).fetchall()

In [3]:
mb_owaty = pd.DataFrame(owaty_mb)
mb_owaty

,NAZWA_UKL,OPIS,RODZAJ_CIECIA,ZUZYCIE,TYP_OWATY
0,O3-AMA-25-SP,"AMALFI 2,5",K,3.54,O3
1,O3-AMA-XLA-SP,AMALFI [LA = LA],K,4.38,O3
2,O3-AMA-XRS-SP,AMALFI [RS = RS],K,6.04,O3
3,O3-AMA-X-SP,AMALFI ][,K,0.96,O3
4,O3-AMA-15-SP,"AMALFI 3,5",K,3.54,O3
...,...,...,...,...,...
470,W3,GOYA P112x80,R,2.50,W3
471,O1-OVA-X1X-SP,OVAL [1o],K,2.10,O1
472,O1-LEN-X1X-SP,LENOX [1o],K,1.90,O1
473,O1-LEN-X1HX-SP,LENOX [1H],K,1.66,O1


In [4]:
def dodaj_KJ(zamowione_paczki):
    czas_calkowity = 0
    kajoty = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        kajoty.append([f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]}", kj_paczki(row[2])*10])
        
        print(f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]},", kj_paczki(row[2])*10)
        czas_calkowity += kj_paczki(row[2])*10
    
    print("Całkowity czas KJ:",czas_calkowity)
    return kajoty

In [5]:
def dodaj_OWATY_ROZKROJ(owaty):
    
    for row in owaty:
        # print(row[0], row[1], row[2], row[3], row[4])
        czas_ciecia = int(mb_owaty[mb_owaty.OPIS == row[1]].ZUZYCIE.sum() * 1.5)
        print(f"WYCINANIE OWAT NR PARTII {row[3]}: {row[1]} - {row[2]}szt| KOMPLETACJA {row[0]},", czas_ciecia)
 

In [6]:
def dodaj_pakowanie(zamowione_paczki):
    czas_calkowity = 0
    pakowanie = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        print(f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]},", row[2])
        czas_calkowity += row[2]
        pakowanie.append([f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]}", row[2]])
    
    print("Całkowity czas PAKOWANIA:",czas_calkowity)

    return pakowanie

In [7]:
suma_paczek

490

In [8]:
zamowione_paczki

[('4_24', 'STONE NW140x150', 50, '24/0660', '', '26/02'),
 ('4_24', 'STONE NW140x150 LEN', 30, '24/0660', '', '26/02'),
 ('4_24', 'STONE WN150x140 LEN', 30, '24/0660', '', '26/02'),
 ('4_24', 'STONE [LH', 20, '24/0660', '', '26/02'),
 ('4_24', 'STONE P106x82', 20, '24/0660', '', '26/02'),
 ('4_24', 'STONE ][', 100, '24/0660', '', '26/02'),
 ('3_24', 'WILLOW 1', 40, '24/0660', '', '26/02'),
 ('3_24', 'WILLOW 1,5', 40, '24/0660', '', '26/02'),
 ('3_24', 'WILLOW [L2Hz', 20, '24/0660', '', '26/02'),
 ('3_24', 'WILLOW [', 80, '24/0660', '', '26/02'),
 ('3_24', 'WILLOW ]', 60, '24/0660', '', '26/02')]

In [9]:
pakowanie = dodaj_pakowanie(zamowione_paczki)

PAKOWANIE PARTII 26/02: STONE NW140x150 - 50| KOMPLETACJA 4_24, 50
PAKOWANIE PARTII 26/02: STONE NW140x150 LEN - 30| KOMPLETACJA 4_24, 30
PAKOWANIE PARTII 26/02: STONE WN150x140 LEN - 30| KOMPLETACJA 4_24, 30
PAKOWANIE PARTII 26/02: STONE [LH - 20| KOMPLETACJA 4_24, 20
PAKOWANIE PARTII 26/02: STONE P106x82 - 20| KOMPLETACJA 4_24, 20
PAKOWANIE PARTII 26/02: STONE ][ - 100| KOMPLETACJA 4_24, 100
PAKOWANIE PARTII 26/02: WILLOW 1 - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 26/02: WILLOW 1,5 - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 26/02: WILLOW [L2Hz - 20| KOMPLETACJA 3_24, 20
PAKOWANIE PARTII 26/02: WILLOW [ - 80| KOMPLETACJA 3_24, 80
PAKOWANIE PARTII 26/02: WILLOW ] - 60| KOMPLETACJA 3_24, 60
Całkowity czas PAKOWANIA: 490


In [10]:
kjoty = dodaj_KJ(zamowione_paczki)

KJ PARTII 26/02: STONE NW140x150 - 50| NR_DOS 24/0660, 50
KJ PARTII 26/02: STONE NW140x150 LEN - 30| NR_DOS 24/0660, 30
KJ PARTII 26/02: STONE WN150x140 LEN - 30| NR_DOS 24/0660, 30
KJ PARTII 26/02: STONE [LH - 20| NR_DOS 24/0660, 20
KJ PARTII 26/02: STONE P106x82 - 20| NR_DOS 24/0660, 20
KJ PARTII 26/02: STONE ][ - 100| NR_DOS 24/0660, 70
KJ PARTII 26/02: WILLOW 1 - 40| NR_DOS 24/0660, 40
KJ PARTII 26/02: WILLOW 1,5 - 40| NR_DOS 24/0660, 40
KJ PARTII 26/02: WILLOW [L2Hz - 20| NR_DOS 24/0660, 20
KJ PARTII 26/02: WILLOW [ - 80| NR_DOS 24/0660, 60
KJ PARTII 26/02: WILLOW ] - 60| NR_DOS 24/0660, 50
Całkowity czas KJ: 430


In [11]:

kjoty

[['KJ PARTII 26/02: STONE NW140x150 - 50| NR_DOS 24/0660', 50],
 ['KJ PARTII 26/02: STONE NW140x150 LEN - 30| NR_DOS 24/0660', 30],
 ['KJ PARTII 26/02: STONE WN150x140 LEN - 30| NR_DOS 24/0660', 30],
 ['KJ PARTII 26/02: STONE [LH - 20| NR_DOS 24/0660', 20],
 ['KJ PARTII 26/02: STONE P106x82 - 20| NR_DOS 24/0660', 20],
 ['KJ PARTII 26/02: STONE ][ - 100| NR_DOS 24/0660', 70],
 ['KJ PARTII 26/02: WILLOW 1 - 40| NR_DOS 24/0660', 40],
 ['KJ PARTII 26/02: WILLOW 1,5 - 40| NR_DOS 24/0660', 40],
 ['KJ PARTII 26/02: WILLOW [L2Hz - 20| NR_DOS 24/0660', 20],
 ['KJ PARTII 26/02: WILLOW [ - 80| NR_DOS 24/0660', 60],
 ['KJ PARTII 26/02: WILLOW ] - 60| NR_DOS 24/0660', 50]]

In [12]:
##eliza 5
##adamr 4
##piotrl 8


mip_session.add(Procesy_Przydzielone(4, 5, 1, "DOSTAWA PARTII 26/02: NR SAM PIANPOL 17_24", "2024-08-09", 620))

for kj in kjoty:
    mip_session.add(Procesy_Przydzielone(4, 5, 2, kj[0], "2024-08-09", kj[1]))

for p in pakowanie:
    mip_session.add(Procesy_Przydzielone(4, 5, 3, p[0], "2024-08-09", p[1]))

# mip_session.add(Procesy_Przydzielone(4, 5, 11, "DOSTAWA OWAT Z:50, N:20, C:80", "2024-08-05", 150*9))

# for p in owaty:

#     mip_session.add(Procesy_Przydzielone(4, 5, 9, p[0], "2024-08-02", p[1]))


mip_session.commit()

In [13]:
mip_session.execute(text("SELECT uid, kid, proces, nazwa_procesu, data_utworzenia, planowany_dzien_rozpoczecia, priorytet, status, aktywna, uwagi_kier, preferowany_czas_wykonania FROM baza_mip.procesy_przydzielone where nazwa_procesu like '%PARTII 26/02%'")).all()

[(4, 5, 9, 'WYCINANIE OWAT: NR PARTII 26/02 NR KOMPL 3_24| WILLOW 1,5 - 40szt', '2024-07-10', '2024-07-10', 1, 3, 1, None, None),
 (4, 5, 9, 'WYCINANIE OWAT NR PARTII 26/02: WILLOW 1 - 40szt| KOMPLETACJA 3_24', '2024-08-02', '2024-08-02', 1, 0, 1, None, 2),
 (4, 5, 9, 'WYCINANIE OWAT NR PARTII 26/02: WILLOW 1,5 - 40szt| KOMPLETACJA 3_24', '2024-08-02', '2024-08-02', 1, 0, 1, None, 3),
 (4, 5, 9, 'WYCINANIE OWAT NR PARTII 26/02: WILLOW [L2Hz - 20szt| KOMPLETACJA 3_24', '2024-08-02', '2024-08-02', 1, 0, 1, None, 14),
 (4, 5, 9, 'WYCINANIE OWAT NR PARTII 26/02: WILLOW [ - 80szt| KOMPLETACJA 3_24', '2024-08-02', '2024-08-02', 1, 0, 1, None, 3),
 (4, 5, 9, 'WYCINANIE OWAT NR PARTII 26/02: WILLOW ] - 60szt| KOMPLETACJA 3_24', '2024-08-02', '2024-08-02', 1, 0, 1, None, 0),
 (4, 5, 1, 'DOSTAWA PARTII 26/02: NR SAM PIANPOL 17_24', '2024-08-08', '2024-08-09', 1, 0, 1, None, 620),
 (4, 5, 2, 'KJ PARTII 26/02: STONE NW140x150 - 50| NR_DOS 24/0660', '2024-08-08', '2024-08-09', 1, 0, 1, None, 50),
 